In [1]:
#Import libraries
import pyam
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np

<IPython.core.display.Javascript object>

In [2]:
def get_data():
    #get IIASA identification
    #workplace IIASA
    iiasa_creds = r"C:\Users\scheifinger\Documents\GitHub\Feasibility_Tool\iiasa_credentials.yml"
    #workplace home
    #iiasa_creds = r"C:\Users\schei\OneDrive\Dokumente\GitHub\Feasibility_Tool\iiasa_credentials.yml"
    pyam.iiasa.Connection(creds=iiasa_creds)

    connections = list(pyam.iiasa.Connection(creds=iiasa_creds).valid_connections)
    #other variables: 'Emissions|CO2', 'Primary Energy|Coal', 
    #query for climate scenario data
    
    df = pyam.read_iiasa(
        name = 'engage_internal',
        creds = iiasa_creds,
        scenario =[
            "T34_1000_ref",
            "T34_1000_govem",
            "T34_1000_feas_em",
            "T34_1000_bitb_em",
            "T34_1000_bitb_ref",
            "T34_1000_enab_em",
            "T34_NPi2100",
            "T34_NDC2100"
        ],
        variable=["Emissions|CO2", 
                  'Emissions|Kyoto Gases',
                  'Primary Energy|Coal',
                  'Primary Energy|Gas',
                  'Primary Energy',
                  'Capacity|Electricity|Coal',
                  'Capacity|Electricity|Solar',
                  'Capacity|Electricity|Wind',
                  'Carbon Sequestration|CCS',
                  'Carbon Sequestration|Land Use',
                  'Secondary Energy|Electricity|Coal',
                  'Secondary Energy|Electricity|Solar'
                  ],
        region=["World",
                "North America; primarily the United States of America and Canada",
                "Eastern and Western Europe (i.e., the EU28)",
                "Pacific OECD",
                "Countries of centrally-planned Asia; primarily China",
                "Countries of South Asia; primarily India",
                "Other countries of Asia",
                "Countries of Sub-Saharan Africa",
                "Countries of Latin America and the Caribbean",
                "Countries of the Middle East; Iran, Iraq, Israel, Saudi Arabia, Qatar, etc.",
                "Reforming Economies of Eastern Europe and the Former Soviet Union; primarily Russia"]
    )   
    #return data format of df
    return df


In [3]:
data_set = get_data().data

pyam - INFO: Running in a notebook, setting up a basic logging at level INFO
pyam.iiasa - INFO: You are connected as user `scheifinger`
httpx - INFO: HTTP Request: POST https://api.manager.ece.iiasa.ac.at/v1/token/obtain/ "HTTP/1.1 200 OK"
pyam.iiasa - INFO: You are connected as user `scheifinger`
httpx - INFO: HTTP Request: GET https://api.manager.ece.iiasa.ac.at/legacy/applications "HTTP/1.1 200 OK"
httpx - INFO: HTTP Request: POST https://api.manager.ece.iiasa.ac.at/v1/token/obtain/ "HTTP/1.1 200 OK"
httpx - INFO: HTTP Request: GET https://api.manager.ece.iiasa.ac.at/legacy/applications "HTTP/1.1 200 OK"
httpx - INFO: HTTP Request: GET https://api.manager.ece.iiasa.ac.at/legacy/applications/IXSE_ENGAGE_INTERNAL/config "HTTP/1.1 200 OK"
pyam.iiasa - INFO: You are connected to the IXSE_ENGAGE_INTERNAL scenario explorer hosted by IIASA. If you use this data in any published format, please cite the data as provided in the explorer guidelines: https://data.ece.iiasa.ac.at/engage-internal

In [4]:
print("Sum of all countries is larger than World?")
#TODO: ask/ check
print(
data_set[(data_set["model"] == "WITCH 5.0") \
    & (data_set['scenario'] == "T34_1000_bitb_em") \
    & (data_set["variable"] == "Primary Energy|Gas") \
    & (data_set["year"] == 2020) \
    & (data_set["region"] != "World")
    ].agg({'value':"sum"})
)

data_set[(data_set["model"] == "WITCH 5.0") \
    & (data_set['scenario'] == "T34_1000_bitb_em") \
    & (data_set["variable"] == "Primary Energy|Gas") \
    & (data_set["year"] == 2020) \
    & (data_set["region"] == 'World')
    ]

print(
    data_set[(data_set['scenario'] == "T34_1000_bitb_em") \
    & (data_set["variable"] == "Primary Energy|Gas") \
    & (data_set["year"] == 2020) \
    & (data_set["region"] != "World")
    ].groupby("model").agg({'value':"sum"})
)

print(
    data_set[(data_set['scenario'] == "T34_1000_bitb_em") \
    & (data_set["variable"] == "Primary Energy|Gas") \
    & (data_set["year"] == 2020) \
    & (data_set["region"] == "World")
    ].groupby("model").agg({'value':"sum"})
    )



Sum of all countries is larger than World?
value    131.927354
dtype: float64
                            value
model                            
AIM/CGE V2.2           140.111100
GEM-E3_V2023           124.479440
IMAGE 3.2              125.988247
MESSAGEix-GLOBIOM_1.1  139.348818
POLES ENGAGE           139.420928
REMIND 3.0             128.937054
WITCH 5.0              131.927354
                            value
model                            
AIM/CGE V2.2           140.111000
GEM-E3_V2023           143.410610
IMAGE 3.2              125.988203
MESSAGEix-GLOBIOM_1.1  139.715211
POLES ENGAGE           144.365265
REMIND 3.0             134.583383
WITCH 5.0              131.927354


In [58]:
#Data wrangling
df = data_set
#get regional groupings
## OECD: "North America; primarily the United States of America and Canada","Eastern and Western Europe (i.e., the EU28)"
#TODO Clarify that countries of Asia and Latin America are missing
## China:  Countries of centrally-planned Asia; primarily China
## Rest of the world: other countries
#get world
world = data_set[df['region'] == "World"]
world.loc[:, "region"] = "World"
#get OECD*
oecd = df[df["region"].isin(["North America; primarily the United States of America and Canada","Eastern and Western Europe (i.e., the EU28)", "Pacific OECD"])]\
    .groupby(["model", "scenario", "variable", "year", "unit"])\
        .agg({"value": "sum"}).reset_index()
oecd['region'] = "OECD"
#get China
china = df[df['region'] == "Countries of centrally-planned Asia; primarily China"]
china.loc[:, "region"] = "China"
#get RoW
row = df[~df["region"].isin(["World", "North America; primarily the United States of America and Canada","Eastern and Western Europe (i.e., the EU28)", "Pacific OECD", "Countries of centrally-planned Asia; primarily China"])]\
        .groupby(["model", "scenario", "variable", "year", "unit"])\
            .agg({"value": "sum"}).reset_index()
row["region"] = "RoW"
#Concat four regions
df = pd.concat([world, oecd, china, row]).reset_index()

#from long to wide
df = pd.pivot(data=df, index=['model','scenario', 'region', 'year'], columns = 'variable', values = 'value').reset_index()
#get diffeernt scenario narratives from scenario name
df['scenario_narrative'] = np.where(df['scenario'].str.contains("T34_1000_ref", case=True), "Cost Effective", np.nan) 
df['scenario_narrative'] = np.where(df['scenario'].str.contains("T34_1000_bitb_ref", case=True), "Tech", df['scenario_narrative']) 
df['scenario_narrative'] = np.where(df['scenario'].str.contains("T34_1000_govem", case=True), "Instit", df['scenario_narrative'])
df['scenario_narrative'] = np.where(df['scenario'].str.contains("T34_1000_bitb_em", case=True), "Tech+Inst", df['scenario_narrative'])
df['scenario_narrative'] = np.where(df['scenario'].str.contains("T34_1000_enab_em", case=True), "Inst+Enab", df['scenario_narrative'])
df['scenario_narrative'] = np.where(df['scenario'].str.contains("T34_1000_feas_em", case=True), "Tech+Inst+Enab", df['scenario_narrative'])
df['scenario_narrative'] = np.where(df['scenario'].str.contains("T34_NDC2100", case=True), "NDC", df['scenario_narrative'])
df['scenario_narrative'] = np.where(df['scenario'].str.contains("T34_NPi2100", case=True), "Current Policy", df['scenario_narrative'])


#caluclate percentages of energymix for coal and gas
df["Share_Coal"] = df["Primary Energy|Coal"] / df["Primary Energy"]
df["Share_Gas"] = df["Primary Energy|Gas"] / df["Primary Energy"]

#get reductions from 2020 - 2040
df.head(3)
#Sort dataframe
#df.sort_values(by=["model", "scenario", "region", "year"], ascending=True, inplace=True)

variable,model,scenario,region,year,Capacity|Electricity|Coal,Capacity|Electricity|Solar,Capacity|Electricity|Wind,Carbon Sequestration|CCS,Carbon Sequestration|Land Use,Emissions|CO2,Emissions|Kyoto Gases,Primary Energy,Primary Energy|Coal,Primary Energy|Gas,Secondary Energy|Electricity|Coal,Secondary Energy|Electricity|Solar,scenario_narrative,Share_Coal,Share_Gas
0,AIM/CGE V2.2,T34_1000_bitb_em,China,2010,773.8319,1.4445,14.6534,0.0,NaN,8860.3168,11537.5187,108.3570,76.8021,4.0615,11.5723,0.0087,Tech+Inst,0.708788,0.037483
1,AIM/CGE V2.2,T34_1000_bitb_em,China,2015,1052.4402,66.0579,61.9462,0.0,NaN,10778.5587,13745.7169,132.7931,92.2560,5.6674,15.0383,0.3993,Tech+Inst,0.694735,0.042678
2,AIM/CGE V2.2,T34_1000_bitb_em,China,2020,1418.3453,343.1131,153.9295,0.0,NaN,11958.4702,15070.7563,151.2920,97.7011,9.5000,18.8428,2.0741,Tech+Inst,0.645778,0.062792


In [62]:
#filter
to_plot_df = df[(df['year'].isin([2020, 2030, 2040, 2050])) & (df["scenario"].isin(["T34_1000_ref", "T34_1000_govem","T34_NPi2100","T34_NDC2100"]))]

#calculate reductions
reductions_df = to_plot_df[["model", "scenario", "scenario_narrative", "region", 'year', "Emissions|CO2"]]
reductions_df = pd.pivot(data=reductions_df, index=reductions_df.drop(['year', "Emissions|CO2"], axis=1).columns, columns = 'year', values = 'Emissions|CO2').reset_index()
reductions_df["2030_CO2_redu"] = ( (reductions_df[2030] - reductions_df[2020]) / reductions_df[2020]) * -1 #multiplied with -1 to have positive values 
reductions_df["2040_CO2_redu"] = ((reductions_df[2040] - reductions_df[2020]) / reductions_df[2020]) * -1 
reductions_df.drop([2020, 2030, 2040], axis=1, inplace=True)

#calculate capacities/usage for 2030
coal_use_2030 = pd.pivot(data=to_plot_df, index=['model','scenario', 'region'], columns = 'year', values = 'Secondary Energy|Electricity|Coal').reset_index()[['model', 'scenario', 'region', 2030]]
coal_use_2030.rename(columns={2030:"coal_use_2030"}, inplace=True)

solar_use_2030 = pd.pivot(data=to_plot_df, index=['model','scenario', 'region'], columns = 'year', values = 'Secondary Energy|Electricity|Solar').reset_index()[['model', 'scenario', 'region', 2030]]
solar_use_2030.rename(columns={2030:"solar_use_2030"}, inplace=True)

ccs_use_2050 = pd.pivot(data=to_plot_df, index=['model','scenario', 'region'], columns = 'year', values = 'Carbon Sequestration|CCS').reset_index()[['model', 'scenario', 'region', 2050]]
ccs_use_2050.rename(columns={2050:"ccs_use_2050"}, inplace=True) #unit 	Mt CO2/yr

	

#merge
to_plot_df = pd.merge(left=reductions_df, right=coal_use_2030, on=["model", "scenario", "region"])
to_plot_df = pd.merge(left=to_plot_df, right=solar_use_2030, on=["model", "scenario", "region"])
to_plot_df = pd.merge(left=to_plot_df, right=ccs_use_2050, on=["model", "scenario", "region"])


# #make CO2 reduction longer for plotting purposes
#reduce data
to_plot_df = pd.melt(to_plot_df, id_vars=["model", 'scenario', 'region', 'scenario_narrative', 'coal_use_2030','solar_use_2030', 'ccs_use_2050'],
                    value_vars=["2030_CO2_redu", "2040_CO2_redu"],
                    var_name='reduction_year', value_name='reduction_value')
#TODO: Note: COFFEE doesn't have govem scenario
to_plot_df

,model,scenario,region,scenario_narrative,coal_use_2030,solar_use_2030,ccs_use_2050,reduction_year,reduction_value
0,AIM/CGE V2.2,T34_1000_govem,China,Instit,15.890800,10.623400,3604.438500,2030_CO2_redu,0.190899
1,AIM/CGE V2.2,T34_1000_govem,OECD,Instit,3.153600,8.215100,4728.317000,2030_CO2_redu,0.349371
2,AIM/CGE V2.2,T34_1000_govem,RoW,Instit,8.802100,12.543600,1313.795600,2030_CO2_redu,0.156149
3,AIM/CGE V2.2,T34_1000_govem,World,Instit,27.846500,31.382000,9646.551200,2030_CO2_redu,0.218336
4,AIM/CGE V2.2,T34_1000_ref,China,Cost Effective,14.993200,11.410800,3207.475000,2030_CO2_redu,0.214562
...,...,...,...,...,...,...,...,...,...
219,WITCH 5.0,T34_NDC2100,World,NDC,26.703312,22.055595,629.500912,2040_CO2_redu,0.174320
220,WITCH 5.0,T34_NPi2100,China,Current Policy,16.604370,2.906712,0.000052,2040_CO2_redu,0.032554
221,WITCH 5.0,T34_NPi2100,OECD,Current Policy,8.304743,8.329264,0.000466,2040_CO2_redu,0.203401
222,WITCH 5.0,T34_NPi2100,RoW,Current Policy,11.154327,4.381168,0.000421,2040_CO2_redu,-0.035808


In [28]:
# GLOBAL 2030
filter_df = to_plot_df[to_plot_df["region"] == "World"]
fig = go.Figure(px.strip(
    filter_df[(to_plot_df['reduction_year'] == '2030_CO2_redu')],#filter because I only need one row per model and scenario
    x='scenario_narrative',
    y='reduction_value',
    color='model',
    stripmode='overlay'))


fig.add_trace(go.Box(
    y = filter_df[(to_plot_df['reduction_year'] == '2030_CO2_redu') & (to_plot_df["scenario_narrative"] == "Instit")]["reduction_value"],
    name = "Instit",
    marker_color='grey',
    opacity=0.5,
    boxpoints=False,
    showlegend=False
))

fig.add_trace(go.Box(
    y = filter_df[(to_plot_df['reduction_year'] == '2030_CO2_redu') & (to_plot_df["scenario_narrative"] == "Cost Effective")]["reduction_value"],
    name = "Cost Effective",
    marker_color='grey',
    opacity=0.3,
    boxpoints=False,
    showlegend=False
))

fig.update_traces({'marker':{'size': 8}})

fig.update_layout(autosize=False,
                  width=600,
                  height=600,
                  title = go.layout.Title(
                    text="ENGAGE 2C scenarios <br><sup>Global CO2 reductions by 2030</sup>",
                    xref="paper",
                    x=0),
                  xaxis_title="Scenario Narrative",
                  yaxis_title = "CO2 reductions",
                  yaxis_tickformat='.0%')
    
fig

C:\Users\scheifinger\AppData\Local\Temp\ipykernel_4416\2420778190.py:4: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\scheifinger\AppData\Local\Temp\ipykernel_4416\2420778190.py:12: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\scheifinger\AppData\Local\Temp\ipykernel_4416\2420778190.py:21: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [38]:
# GLOBAL 2030 + Current policies
filter_df = to_plot_df[to_plot_df["region"] == "World"]
fig = go.Figure(px.strip(
    filter_df[(to_plot_df['reduction_year'] == '2030_CO2_redu')],#filter because I only need one row per model and scenario
    x='scenario_narrative',
    y='reduction_value',
    color='model',
    stripmode='overlay'))


fig.add_trace(go.Box(
    y = filter_df[(to_plot_df['reduction_year'] == '2030_CO2_redu') & (to_plot_df["scenario_narrative"] == "Instit")]["reduction_value"],
    name = "Instit",
    marker_color='grey',
    opacity=0.5,
    boxpoints=False,
    showlegend=False
))

fig.add_trace(go.Box(
    y = filter_df[(to_plot_df['reduction_year'] == '2030_CO2_redu') & (to_plot_df["scenario_narrative"] == "Cost Effective")]["reduction_value"],
    name = "Cost Effective",
    marker_color='grey',
    opacity=0.3,
    boxpoints=False,
    showlegend=False
))

fig.add_trace(go.Box(
    y = filter_df[(to_plot_df['reduction_year'] == '2030_CO2_redu') & (to_plot_df["scenario_narrative"] == "NDC")]["reduction_value"],
    name = "NDC",
    marker_color='grey',
    opacity=0.3,
    boxpoints=False,
    showlegend=False
))

fig.add_trace(go.Box(
    y = filter_df[(to_plot_df['reduction_year'] == '2030_CO2_redu') & (to_plot_df["scenario_narrative"] == "Current Policy")]["reduction_value"],
    name = "Current Policy",
    marker_color='grey',
    opacity=0.3,
    boxpoints=False,
    showlegend=False
))

fig.update_traces({'marker':{'size': 8}})

fig.update_layout(autosize=False,
                  width=600,
                  height=600,
                  title = go.layout.Title(
                    text="ENGAGE 2C scenarios <br><sup>Global CO2 reductions by 2030</sup>",
                    xref="paper",
                    x=0),
                  xaxis_title="Scenario Narrative",
                  yaxis_title = "CO2 reductions",
                  yaxis_tickformat='.0%')
    
fig

C:\Users\scheifinger\AppData\Local\Temp\ipykernel_4416\2741664121.py:4: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\scheifinger\AppData\Local\Temp\ipykernel_4416\2741664121.py:12: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\scheifinger\AppData\Local\Temp\ipykernel_4416\2741664121.py:21: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\scheifinger\AppData\Local\Temp\ipykernel_4416\2741664121.py:30: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\scheifinger\AppData\Local\Temp\ipykernel_4416\2741664121.py:39: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [22]:
# GLOBAL 2040
fig = go.Figure(px.strip(
    to_plot_df[(to_plot_df['year'] == 2030)],#filter because I only need one row per model and scenario
    x='scenario_narrative',
    y='2040_CO2_redu',
    color='model',
    stripmode='overlay'))


fig.add_trace(go.Box(
    y = to_plot_df[(to_plot_df['year'] == 2030) & (to_plot_df["scenario_narrative"] == "Instit")]["2040_CO2_redu"],
    name = "Instit",
    marker_color='grey',
    opacity=0.5,
    boxpoints=False,
    showlegend=False
))

fig.add_trace(go.Box(
    y = to_plot_df[(to_plot_df['year'] == 2030) & (to_plot_df["scenario_narrative"] == "Cost Effective")]["2040_CO2_redu"],
    name = "Cost Effective",
    marker_color='grey',
    opacity=0.3,
    boxpoints=False,
    showlegend=False
))

fig.update_traces({'marker':{'size': 8}})

fig.update_layout(autosize=False,
                  width=600,
                  height=600,
                  title = go.layout.Title(
                    text="ENGAGE 2C scenarios <br><sup>Global CO2 reductions by 2040</sup>",
                    xref="paper",
                    x=0),
                  xaxis_title="Scenario Narrative",
                  yaxis_title = "CO2 reductions",
                  yaxis_tickformat='.0%')
    
fig

KeyError: 'year'

In [63]:
#Feasibility constraints

#Coal reduction
#get media coal consumption in 2020
df[(df["year"] == 2030)].groupby(["region"])["Secondary Energy|Electricity|Coal"].describe()

#try filtering
min_coal_use_2030 = 1
max_solar_use_2030 = 40
#filter_df = pd.pivot(data=to_plot_df, index=['model','scenario', 'region', 'scenario_narrative', '2030_CO2_redu', '2040_CO2_redu'], columns = 'year', values = 'Secondary Energy|Electricity|Coal').reset_index()
filter_df = to_plot_df[(to_plot_df['coal_use_2030'] >= min_coal_use_2030) &\
                        (to_plot_df['solar_use_2030'] <= max_solar_use_2030) &\
                        (to_plot_df['region'] != "World")]




df[(df["year"] == 2030)].groupby(["region"])["Secondary Energy|Electricity|Coal"].describe()

df[(df["year"] == 2030)].groupby(["model"])["Secondary Energy|Electricity|Solar"].describe()
#to_plot_df[to_plot_df["region"] == "OECD"].groupby(['model'])[["coal_use_2030","solar_use_2030"]].min().min()
to_plot_df.groupby(['region'])[["solar_use_2030", "coal_use_2030", "ccs_use_2050"]].describe()
# px.line(data_frame= df[(df["model"].isin(["IMAGE 3.2", "WITCH 5.0"])) & (df['region'] == "World") & (df["scenario"].isin(["T34_1000_ref", "T34_1000_govem"]))],
#         x="year",
#         y="Emissions|CO2",
#         color="model",
#         markers="scenario"     
# )


solar_use_2030                                                        
                count       mean       std       min        25%        50%   
region                                                                       
China            56.0   5.838600  3.458289  1.079159   3.147270   5.274653  \
OECD             56.0   6.342230  2.974541  1.895875   4.375460   5.626193   
RoW              56.0   6.309051  4.639789  0.606488   2.955446   4.588248   
World            56.0  18.800685  9.823043  3.618173  11.121933  16.828754   

                             coal_use_2030             ...              
              75%        max         count       mean  ...        75%   
region                                                 ...              
China    8.107662  12.407217          56.0  14.255353  ...  17.985347  \
OECD     8.433792  13.553495          56.0   3.315698  ...   4.176373   
RoW      9.420500  15.910465          56.0   7.589699  ...   9.323534   
World   23.656286  39.180369          56.0  25.426672  ...  31.267411   

                  ccs_use_2050                                              
              max        count         mean          std  min         25%   
region                                                                      
China   24.824760         44.0   949.398214  1102.450475  0.0    0.018008  \
OECD    13.353713         46.0  1331.532249  1213.009272  0.0  312.253761   
RoW     13.318324         46.0  1572.089059  1756.770959  0.0  138.461063   
World   45.370248         46.0  3909.863046  3557.356190  0.0  649.691817   

                                                
                50%          75%           max  
region                                          
China    777.892720  1515.210945   3604.438500  
OECD    1080.914251  1838.331048   4728.317000  
RoW     1334.286908  1923.678439   7305.685230  
World   3434.162067  5236.769354  12038.996601  

[4 rows x 24 columns]

In [9]:
#Scenario Space
#explore full scenario space
##get min and max values for all regions across all scenarios
min = to_plot_df.groupby(['region'])[["solar_use_2030", "coal_use_2030"]].min().reset_index()
min.rename(columns = {'solar_use_2030': "MIN_solar_use_2030", 'coal_use_2030': 'MIN_coal_use_2030'}, inplace = True)
max = to_plot_df.groupby(['region'])[["solar_use_2030", "coal_use_2030"]].max().reset_index()
max.rename(columns = {'solar_use_2030': "MAX_solar_use_2030", 'coal_use_2030': 'MAX_coal_use_2030'}, inplace = True)



pd.merge(left=min, right=max, on="region").reindex(columns=[
    'region',
    'MIN_solar_use_2030',
    'MAX_solar_use_2030',
    'MIN_coal_use_2030',
    'MAX_coal_use_2030'
])


,region,MIN_solar_use_2030,MAX_solar_use_2030,MIN_coal_use_2030,MAX_coal_use_2030
0,China,1.079159,12.407217,3.564727e-07,19.725314
1,OECD,2.444933,13.553495,2.975541e-01,7.548869
2,RoW,1.278904,15.910465,7.563150e-01,9.916576
3,World,4.972892,39.180369,2.014303e+00,31.663743


In [57]:
test_coal = 0
test_ccs = 1400
test_region = "World"


print(str(len(to_plot_df[(to_plot_df["region"] == test_region) & (to_plot_df["ccs_use_2030"] <= test_ccs)].groupby('model').count()['scenario'])) + "/8 models")
print(str(to_plot_df[(to_plot_df["region"] == test_region) & (to_plot_df["ccs_use_2030"] <= test_ccs)].groupby('model').count().sum().scenario) + "/30 scenarios")
print(to_plot_df[(to_plot_df["region"] == test_region) & (to_plot_df["ccs_use_2030"] <= test_ccs)].groupby('model').count()['scenario'])

to_plot_df[(to_plot_df["model"] == "POLES ENGAGE") & (to_plot_df["reduction_year"] == "2030_CO2_redu")]

7/8 models
46/30 scenarios
model
AIM/CGE V2.2             6
COFFEE 1.5               4
GEM-E3_V2023             8
IMAGE 3.2                8
MESSAGEix-GLOBIOM_1.1    4
REMIND 3.0               8
WITCH 5.0                8
Name: scenario, dtype: int64


,model,scenario,region,scenario_narrative,coal_use_2030,solar_use_2030,ccs_use_2030,reduction_year,reduction_value
64,POLES ENGAGE,T34_1000_govem,China,Instit,12.550456,11.327579,NaN,2030_CO2_redu,0.203070
65,POLES ENGAGE,T34_1000_govem,OECD,Instit,1.548296,5.984885,NaN,2030_CO2_redu,0.447092
66,POLES ENGAGE,T34_1000_govem,RoW,Instit,8.552605,4.795329,NaN,2030_CO2_redu,0.085193
67,POLES ENGAGE,T34_1000_govem,World,Instit,22.770554,22.240467,NaN,2030_CO2_redu,0.200665
68,POLES ENGAGE,T34_1000_ref,China,Cost Effective,13.413551,11.105577,NaN,2030_CO2_redu,0.184776
69,POLES ENGAGE,T34_1000_ref,OECD,Cost Effective,2.607351,5.600019,NaN,2030_CO2_redu,0.327067
70,POLES ENGAGE,T34_1000_ref,RoW,Cost Effective,7.885040,4.911810,NaN,2030_CO2_redu,0.171462
71,POLES ENGAGE,T34_1000_ref,World,Cost Effective,24.011316,21.758619,NaN,2030_CO2_redu,0.192391
72,POLES ENGAGE,T34_NDC2100,China,NDC,22.603245,2.669643,NaN,2030_CO2_redu,-0.029902
73,POLES ENGAGE,T34_NDC2100,OECD,NDC,2.726007,5.382869,NaN,2030_CO2_redu,0.346620


In [53]:
##CHECK Policy
px.line(
    data_frame= df[((df['scenario'] == "T34_NDC2100") | (df["scenario"] == "T34_NPi2100")) &(df['region']=="World")]
    ,x="year"
    ,y="Emissions|CO2"
    ,color="scenario"
    ,line_dash="model"
)

df[((df['scenario'] == "T34_NDC2100") | (df["scenario"] == "T34_NPi2100"))].groupby(['model', 'scenario']).describe()
df[((df['scenario'] == "T34_NDC2100") & (df["model"] == "IMAGE 3.2")) & (df["region"] == "World")]

array(['IMAGE 3.2'], dtype=object)

In [24]:
#get boxplot data from NDC & NPi


variable                                   Capacity|Electricity|Coal   
model        scenario         region year                              
AIM/CGE V2.2 T34_1000_bitb_em China  2010                 773.831900  \
                                     2015                1052.440200   
                                     2020                1418.345300   
                                     2025                1627.245700   
                                     2030                1554.853200   
...                                                              ...   
WITCH 5.0    T34_1000_ref     World  2080                 316.943016   
                                     2085                 271.095727   
                                     2090                 231.880464   
                                     2095                 198.337871   
                                     2100                 169.647371   

variable                                   Capacity|Electricity|Solar   
model        scenario         region year                               
AIM/CGE V2.2 T34_1000_bitb_em China  2010                    1.444500  \
                                     2015                   66.057900   
                                     2020                  343.113100   
                                     2025                  869.817100   
                                     2030                 1991.788100   
...                                                               ...   
WITCH 5.0    T34_1000_ref     World  2080                30327.633257   
                                     2085                32499.802703   
                                     2090                33635.986743   
                                     2095                34400.909467   
                                     2100                35108.923937   

variable                                   Capacity|Electricity|Wind   
model        scenario         region year                              
AIM/CGE V2.2 T34_1000_bitb_em China  2010                  14.653400  \
                                     2015                  61.946200   
                                     2020                 153.929500   
                                     2025                 299.663600   
                                     2030                 577.686300   
...                                                              ...   
WITCH 5.0    T34_1000_ref     World  2080                7912.127852   
                                     2085                8411.131342   
                                     2090                8793.346077   
                                     2095                9067.978281   
                                     2100                9273.556347   

variable                                   Carbon Sequestration|CCS   
model        scenario         region year                             
AIM/CGE V2.2 T34_1000_bitb_em China  2010                  0.000000  \
                                     2015                  0.000000   
                                     2020                  0.000000   
                                     2025                  0.000000   
                                     2030                 47.916800   
...                                                             ...   
WITCH 5.0    T34_1000_ref     World  2080              13019.949055   
                                     2085              14488.485935   
                                     2090              15839.578086   
                                     2095              17260.237502   
                                     2100              18563.372220   

variable                                   Carbon Sequestration|Land Use   
model        scenario         region year                                  
AIM/CGE V2.2 T34_1000_bitb_em China  2010                       0.000000  \
                        

# OLD CODE BELOW


In [83]:
# Define color mapping for models
color_mapping = {
    'AIM/CGE V2.2': "rgb(255, 0, 0)",
    'COFFEE 1.5': "rgb(0, 255, 0)",
    'GEM-E3_V2023': "rgb(0, 0, 255)",
    'IMAGE 3.2': "rgb(255, 255, 0)",
    'MESSAGEix-GLOBIOM_1.1': "rgb(255, 0, 255)",
    'POLES ENGAGE': "rgb(0, 255, 255)",
    'REMIND 3.0': "rgb(128, 0, 0)",
    'WITCH 5.0': "rgb(0, 128, 0)"
}

# Map colors based on the "model" column
filter_df["color"] = filter_df["model"].map(color_mapping)

fig = make_subplots(rows=1, cols=len(filter_df["reduction_year"].unique()), shared_yaxes=True)

for i, year in enumerate(filter_df["reduction_year"].unique()):
    box_trace = go.Box(
        x=filter_df[(filter_df["reduction_year"] == year)]["scenario_narrative"],
        y=filter_df[(filter_df["reduction_year"] == year)]["reduction_value"],
        name="Boxplot",
        boxpoints=False,
        marker_color='grey',
        opacity=0.3,
        showlegend=False
    )

    scatter_trace = go.Scatter(
        x=filter_df[(filter_df["reduction_year"] == year)]["scenario_narrative"],
        y=filter_df[(filter_df["reduction_year"] == year)]["reduction_value"],
        mode="markers",
        name="Scatterplot",
        marker=dict(
            color=filter_df[(filter_df["reduction_year"] == year)]["color"],
            size=6
        ),
        showlegend=False,
        legendgroup="Models"
    )

    fig.add_trace(box_trace, row=1, col=i + 1)
    fig.add_trace(scatter_trace, row=1, col=i + 1)

    # Add subplot titles
    fig.update_layout(
        annotations=[dict(text=str(year), xref="x" + str(i + 1), yref="paper", x=0.5, y=1.1, showarrow=False) for i, year in enumerate(filter_df["reduction_year"].unique())]
    )

    # Update legend to show scatterplot colors
# Add legend items for scatterplot colors
for model in filter_df["model"].unique():
    fig.add_trace(go.Scatter(
        x=[None],
        y=[None],
        marker=dict(color=color_mapping[model]),
        name=model,
        legendgroup="Models", 
        showlegend=True
    ))

fig.update_layout(
    title="Boxplot with Scatterplot",
    xaxis=dict(title="Scenario Narrative"),
    yaxis=dict(title="Reduction Value"),
    legend=dict(
        traceorder="normal",
        itemsizing="constant"
    )
)

fig.show()

In [53]:
# Define color mapping for models
color_mapping = {
    'AIM/CGE V2.2': "rgb(255, 0, 0)",
    'COFFEE 1.5': "rgb(0, 255, 0)",
    'GEM-E3_V2023': "rgb(0, 0, 255)",
    'IMAGE 3.2': "rgb(255, 255, 0)",
    'MESSAGEix-GLOBIOM_1.1': "rgb(255, 0, 255)",
    'POLES ENGAGE': "rgb(0, 255, 255)",
    'REMIND 3.0': "rgb(128, 0, 0)",
    'WITCH 5.0': "rgb(0, 128, 0)"
}

# Map colors based on the "model" column
filter_df["color"] = filter_df["model"].map(color_mapping)

fig = make_subplots(rows=1, cols=len(filter_df["reduction_year"].unique()), shared_yaxes=True)

for i, year in enumerate(filter_df["reduction_year"].unique()):
    box_trace = go.Box(
        x=filter_df[(filter_df["reduction_year"] == year)]["scenario_narrative"],
        y=filter_df[(filter_df["reduction_year"] == year)]["reduction_value"],
        name="Boxplot",
        boxpoints=False,
         marker_color='grey',
        opacity=0.3,
        showlegend=False
    )

    fig.add_trace(box_trace, row=1, col=i + 1)

    # Add subplot titles
    fig.update_layout(
        annotations=[dict(text=str(year), xref="x" + str(i + 1), yref="paper", x=0.5, y=1.1, showarrow=False) for i, year in enumerate(filter_df["reduction_year"].unique())]
    )

    # Create a scatterplot for each model
    for model in filter_df["model"].unique():
        scatter_trace = go.Scatter(
            x=filter_df[(filter_df["reduction_year"] == year) & (filter_df["model"] == model)]["scenario_narrative"],
            y=filter_df[(filter_df["reduction_year"] == year) & (filter_df["model"] == model)]["reduction_value"],
            mode="markers",
            name=model,
            marker=dict(
                color=color_mapping[model],
                size=6
            ),
            showlegend=i==0,
            legendgroup=model
        )

        fig.add_trace(scatter_trace, row=1, col=i + 1)

fig.update_layout(
    title = go.layout.Title(
        text="ENGAGE 2C scenarios <br><sup>Global CO2 reductions by 2030 and 2040</sup>",
        xref="paper",
        x=0),
    xaxis=dict(title="Scenario Narrative", ),
    yaxis=dict(title="Reduction Value", range=[0, 0.6]),
    legend=dict(
        traceorder="normal",
        itemsizing="constant"
    )
)

fig.show()

C:\Users\scheifinger\AppData\Local\Temp\ipykernel_13332\2045176072.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [66]:
filter_df = filter_df[filter_df['reduction_year'] == "2030_CO2_redu"]
filter_df

,model,scenario,region,scenario_narrative,coal_use_2030,solar_use_2030,reduction_year,reduction_value,color
0,AIM/CGE V2.2,T34_1000_govem,China,Instit,15.890800,10.623400,2030_CO2_redu,0.190899,"rgb(255, 0, 0)"
1,AIM/CGE V2.2,T34_1000_govem,OECD,Instit,3.153600,8.215100,2030_CO2_redu,0.349371,"rgb(255, 0, 0)"
2,AIM/CGE V2.2,T34_1000_govem,RoW,Instit,8.802100,12.543600,2030_CO2_redu,0.156149,"rgb(255, 0, 0)"
4,AIM/CGE V2.2,T34_1000_ref,China,Cost Effective,14.993200,11.410800,2030_CO2_redu,0.214562,"rgb(255, 0, 0)"
5,AIM/CGE V2.2,T34_1000_ref,OECD,Cost Effective,4.231800,6.472300,2030_CO2_redu,0.206020,"rgb(255, 0, 0)"
6,AIM/CGE V2.2,T34_1000_ref,RoW,Cost Effective,7.840800,14.583000,2030_CO2_redu,0.294489,"rgb(255, 0, 0)"
8,COFFEE 1.5,T34_1000_ref,China,Cost Effective,4.636654,1.079159,2030_CO2_redu,0.318801,"rgb(0, 255, 0)"
9,COFFEE 1.5,T34_1000_ref,OECD,Cost Effective,7.548869,2.578177,2030_CO2_redu,-0.110502,"rgb(0, 255, 0)"
10,COFFEE 1.5,T34_1000_ref,RoW,Cost Effective,2.247348,1.278904,2030_CO2_redu,0.146723,"rgb(0, 255, 0)"
12,GEM-E3_V2023,T34_1000_govem,China,Instit,12.059744,5.712676,2030_CO2_redu,0.123487,"rgb(0, 0, 255)"


In [57]:
#REGIONAL PLOT

#filter data on reduction year
filter_df = filter_df[filter_df['reduction_year'] == "2030_CO2_redu"]
# Define color mapping for models
color_mapping = {
    'AIM/CGE V2.2': "rgb(255, 0, 0)",
    'COFFEE 1.5': "rgb(0, 255, 0)",
    'GEM-E3_V2023': "rgb(0, 0, 255)",
    'IMAGE 3.2': "rgb(255, 255, 0)",
    'MESSAGEix-GLOBIOM_1.1': "rgb(255, 0, 255)",
    'POLES ENGAGE': "rgb(0, 255, 255)",
    'REMIND 3.0': "rgb(128, 0, 0)",
    'WITCH 5.0': "rgb(0, 128, 0)"
}

# Map colors based on the "model" column
filter_df["color"] = filter_df["model"].map(color_mapping)

fig = make_subplots(rows=1, cols=len(filter_df["region"].unique()), shared_yaxes=True)

for i, region_ in enumerate(filter_df["region"].unique()):
    box_trace = go.Box(
        x=filter_df[(filter_df["region"] == region_)]["scenario_narrative"],
        y=filter_df[(filter_df["region"] == region_)]["reduction_value"],
        name="Boxplot",
        boxpoints=False,
        marker_color='grey',
        opacity=0.3,
        showlegend=False
    )

    fig.add_trace(box_trace, row=1, col=i + 1)

    # Add subplot titles
    fig.update_layout(
        annotations=[dict(text=str(year), xref="x" + str(i + 1), yref="paper", x=0.5, y=1.1, showarrow=False) for i, year in enumerate(filter_df["region"].unique())]
    )

    # Create a scatterplot for each model
    for model in filter_df["model"].unique():
        scatter_trace = go.Scatter(
            x=filter_df[(filter_df["region"] == region_) & (filter_df["model"] == model)]["scenario_narrative"],
            y=filter_df[(filter_df["region"] == region_) & (filter_df["model"] == model)]["reduction_value"],
            mode="markers",
            name=model,
            marker=dict(
                color=color_mapping[model],
                size=6
            ),
            showlegend=i==0,
            legendgroup=model
        )

        fig.add_trace(scatter_trace, row=1, col=i + 1)

fig.update_layout(
    title = go.layout.Title(
        text="ENGAGE 2C scenarios <br><sup>Regional CO2 reductions by 2030 </sup>",
        xref="paper",
        x=0),
    xaxis=dict(title="Scenario Narrative", ),
    yaxis=dict(title="Reduction Value", range=[0, 0.6]),
    legend=dict(
        traceorder="normal",
        itemsizing="constant"
    )
)

fig.show()

C:\Users\scheifinger\AppData\Local\Temp\ipykernel_13332\3601570762.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [53]:
fig = go.Figure()

fig = go.Figure()
fig.add_trace(go.Box(
    x=to_plot_df["scenario_narrative"],
    y=to_plot_df["Emissions|CO2"],
    name="All Points",
    jitter=0.3,
    pointpos=0,
    boxpoints='all',  # represent all points
    #marker_color=to_plot_df["scenario_narrative"],
    line_color='rgb(7,40,89)'


))


fig2 = px.box(
    data_frame = to_plot_df
    ,y = 'Emissions|CO2'
    ,x = 'scenario_narrative'
    ,color = "scenario_narrative"
    ,points = 'all'
    ,hover_data = {'model':True, 'scenario_narrative':False}
)
fig2
 

In [122]:
#Figure 2: GHG trajectories
fig2 = px.line(df,
               x = "year",
               y="Emissions|Kyoto Gases",
               color = "region",
               line_dash="model",
               facet_col="scenario_narrative",
               facet_col_wrap= 3,
               range_x=[2005, 2100],
               range_y = [-10000, 20000],
               hover_data = {'scenario':True, 'model':True}, 
               title = "GHG emission trajectories")
fig2.update_layout(showlegend=False)
fig2

In [109]:
#calculate year that each scenario hit's net zero
netzero_df = df[df["Emissions|CO2"] <= 0].groupby(["model", "scenario"])['year'].min().reset_index()
netzero_df.rename(columns={"year": "year_netzero"}, inplace=True)
netzero_df.head()
#aggregated cost until 2100
#df["year"] = pd.to_numeric(df['year'])
#df[df["year"] <= 2100].groupby(["model", "scenario"]).agg({"AR5 climate diagnostics|Temperature|Exceedance Probability|1.5 degC|MAGICC6": ['sum']})

In [32]:
#get amounts of unabated coal
#both Coal data are in EJ/yr
unabcoal_df = df[["model", "scenario", "carbon_budget", "region","year", "Primary Energy|Coal", "Primary Energy|Coal|w/ CCS"]] 
unabcoal_df["Unabated Coal"] = unabcoal_df["Primary Energy|Coal"] - unabcoal_df["Primary Energy|Coal|w/ CCS"]
#pick threshold for when coal is supposed to the phased out // now it's 8EJ/y
unabcoal_df = unabcoal_df[unabcoal_df["Unabated Coal"] <= 8].groupby(["model", "scenario", "carbon_budget"])['year'].min().reset_index()
unabcoal_df.rename(columns={"year": "year_netcoal"}, inplace=True)
#combine with netzero df

unabcoal_df

C:\Users\schei\AppData\Local\Temp\ipykernel_10492\1471908738.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,model,scenario,carbon_budget,year_netcoal
0,AIM/CGE V2.2,T34_1000_feas_em,2C,2055
1,AIM/CGE V2.2,T34_1000_feas_pr,2C,2055
2,AIM/CGE V2.2,T34_1000_feas_ref,2C,2050
3,GEM-E3_V2023,T34_1000_feas_em,2C,2065
4,GEM-E3_V2023,T34_1000_feas_ref,2C,2080
5,IMAGE 3.2,T34_1000_feas_em,2C,2060
6,IMAGE 3.2,T34_1000_feas_pr,2C,2060
7,IMAGE 3.2,T34_1000_feas_ref,2C,2060
8,MESSAGEix-GLOBIOM_1.1,T34_1000_feas_em,2C,2060
9,MESSAGEix-GLOBIOM_1.1,T34_1000_feas_pr,2C,2060


In [89]:
#calculate change in transportation energy from 2020 to 2030
tranred_df = df[["model", "scenario", "carbon_budget", "region","year", "Final Energy|Transportation"]]
#filter for year 2020 & 2030
tranred_df = tranred_df[(tranred_df["year"] == 2020) | (tranred_df["year"] == 2030) ].reset_index().drop("index", axis=1)
#from long to wide
tranred_df = pd.pivot(data=tranred_df, index=["model", "scenario", "carbon_budget", "region"], 
             columns="year", values= "Final Energy|Transportation").reset_index()
#Finale Energy Transportation = (FET)
tranred_df["redu_FET"] = (tranred_df[2030] - tranred_df[2020]) / tranred_df[2020] 

# #both Coal data are in EJ/yr
# unabcoal_df = df[["model", "scenario", "carbon_budget", "region","year", "Primary Energy|Coal", "Primary Energy|Coal|w/ CCS"]] 
# unabcoal_df["Unabated Coal"] = unabcoal_df["Primary Energy|Coal"] - unabcoal_df["Primary Energy|Coal|w/ CCS"]
# #pick threshold for when coal is supposed to the phased out // now it's 8EJ/y
# unabcoal_df = unabcoal_df[unabcoal_df["Unabated Coal"] <= 8].groupby(["model", "scenario", "carbon_budget"])['year'].min().reset_index()
# unabcoal_df.rename(columns={"year": "year_netcoal"}, inplace=True)

px.line(df,
        x = "year",
        y="Final Energy|Transportation",
        color = "carbon_budget",
        line_dash="model",
        facet_col="scenario_narrative",
        hover_data = {'scenario':True, 'model':True})

tranred_df

year,model,scenario,carbon_budget,region,2020,2030,redu_FET
0,AIM/CGE V2.2,T34_1000_feas_em,2C,World,123.601600,104.300200,-0.156158
1,AIM/CGE V2.2,T34_1000_feas_pr,2C,World,123.601600,104.296500,-0.156188
2,AIM/CGE V2.2,T34_1000_feas_ref,2C,World,123.601600,111.827600,-0.095258
3,GEM-E3_V2023,T34_1000_feas_em,2C,World,106.172003,107.829240,0.015609
4,GEM-E3_V2023,T34_1000_feas_ref,2C,World,106.172003,107.223507,0.009904
5,IMAGE 3.2,T34_1000_feas_em,2C,World,92.399406,78.249680,-0.153137
6,IMAGE 3.2,T34_1000_feas_pr,2C,World,92.399406,77.118727,-0.165376
7,IMAGE 3.2,T34_1000_feas_ref,2C,World,92.399406,85.292977,-0.076910
8,MESSAGEix-GLOBIOM_1.1,T34_1000_feas_em,2C,World,113.118992,140.618740,0.243105
9,MESSAGEix-GLOBIOM_1.1,T34_1000_feas_pr,2C,World,113.118992,141.497312,0.250871


In [91]:
#merge reduction data to rest of df
to_plot = pd.merge(left=netzero_df, right=unabcoal_df, on=["model", "scenario", "carbon_budget"])
to_plot = pd.merge(left=to_plot, right=tranred_df.loc[:, ~tranred_df.columns.isin([2020, 2030])],
                   on=["model", "scenario", "carbon_budget"])
to_plot.head(3)

,model,scenario,carbon_budget,year_netzero,year_netcoal,region,redu_FET
0,AIM/CGE V2.2,T34_1000_feas_em,2C,2060,2055,World,-0.156158
1,AIM/CGE V2.2,T34_1000_feas_pr,2C,2055,2055,World,-0.156188
2,AIM/CGE V2.2,T34_1000_feas_ref,2C,2070,2050,World,-0.095258


In [86]:
fig4 = px.scatter(
    data_frame=to_plot,
    y = "redu_FET",
    x="year_netcoal",
    color= "carbon_budget"
)
fig4

In [102]:
#Statement
#Until when is it feasible to phase out coal
year_coal_phaseout = 2050
#What change in energy consumption of transport sector is realistic until 2030?
change_energy_transport = 0

#filter dataframe
filter_df = to_plot.loc[(to_plot["year_netcoal"] >= year_coal_phaseout) & (to_plot["redu_FET"] >= change_energy_transport)]
earliest_net_zero_year = filter_df["year_netzero"].min()
PA_aligned = (filter_data_set["carbon_budget"].str.contains("1.5C").sum() > 0)

print("OUTPUT:")
print("Earliest possible net-zero year:", earliest_net_zero_year)
print("Is PA still possible?", PA_aligned)

OUTPUT:
Earliest possible net-zero year: 2050
Is PA still possible? True


Links:

Plotly Express multiple plots overlay
https://community.plotly.com/t/plotly-express-multiple-plots-overlay/31984
